# Ladder Simulation 2.0
## Goals
 - Create a computer simulation of the Clash Royale ladder
 - Use the simultation to determine the effects of different matchmaking rules
 - Test Card Level based matchmaking
 - Test King tower matchmaking
 - Test no matchmaking rules
 - Change the reset point and reset %
 - Change Inflation rules
        -Reset into arenas

In [1]:
import random
import player as pl
import pandas as pd
import numpy as np
import matplotlib as mpl
from matplotlib import pyplot as plt
import seaborn as sns
%matplotlib inline

0.9

In [4]:
#Global Variables:
COLS = ["ID", "Trophies", "Wins","Losses", "King Tower", "Card Level","Total Level Difference", "LvlDiff/Match"]
PALETTE = sns.color_palette("mako_r", as_cmap=True)

In [5]:
def createArray(numPlayers, trophies):
    """Creates an array of player objects
    Params:
    numPlayers: Integer, number of player objects to be created
    trophies: the starting number of trophies"""
    playerList = [pl.createPlayer(random.choice(range(8,15)), id = i, trophies= trophies) for  i in range(numPlayers)]
    return np.asarray(playerList)

In [6]:
def simulate(numPlayers, initialTrophies, numMatches):
    """Simulates the CR ladder with numPlayers players starting at initialTrophies
    Used to set a baseline for uniform distribution of players across KTs
    plays numMatches matches
    Args:  numPlayers - int, number of players in the ladder
    initialTrophies - int, initial starting numbr of trophies
    numMatches - int, Number of matches to play
    """
    playerArr = createArray(numPlayers, initialTrophies)
    queue = np.asarray([random.choice(playerArr)], dtype = object)
    matchesPlayed = 0
    maxQueueSize = 0
    while matchesPlayed < numMatches:
        if matchesPlayed %(numMatches//10) == 0:
            print(matchesPlayed)
        if queue.size> maxQueueSize:
            maxQueueSize = queue.size
        if queue.size == 0:
            queue = np.append(queue, random.choice(playerArr))
            pass
        else:
            newPlayer = random.choice(playerArr)
            addPos = np.searchsorted(queue, newPlayer)
            if newPlayer in queue:
                pass #can't play against themself
            else:
                addPos = np.searchsorted(queue, newPlayer)
                try:
                    opponent = queue[addPos]
                except IndexError as error:
                    queue = np.insert(queue, addPos, newPlayer)
                    pass
                if opponent.matchAllowed(newPlayer):
                    newPlayer.playMatch(opponent)
                    matchesPlayed += 1
                    queue = np.delete(queue, addPos)
                else:
                    queue = np.insert(queue, addPos, newPlayer)
    playerArr.sort()
    print(f"Max queue size: {maxQueueSize}")
    return playerArr

In [7]:
def continueSim(playerArr, numMatches, mode = None, cardLvlRule = 0, KTdiff = 0, KTcutoff = 5000):
    """Continues a simulation for numMatches more games, Very slow if using mode isn't none
    Args: 
    playerArr: Array of player objects.  
    numMatches: Number of matches to be played
    mode: The type of matchmaking to use.  Either 'KT', 'CL', 'KTCL'
    cardLvlRule: Int, the maximum difference in card levels allowed in a match
    KTdiff: The maximum difference in king tower between both players
    KTcutoff: The maximum trophies where the KT diff rule applies. """
    queue = np.asarray([random.choice(playerArr)], dtype = object)
    matchesPlayed = 0
    cardRule = lambda p1, p2: abs(p1.cardLevel - p2.cardLevel) < cardLvlRule
    kTDiffRule = lambda p1, p2: abs(p1.kt - p2.kt) < KTdiff
    ktCutoffRule = lambda p: p.trophies > KTcutoff
    maxQueueSize = 0
    numMatches += 10 #off sets the matches played counter
    while matchesPlayed < numMatches:

        if matchesPlayed %(numMatches//10) == 0:
            print(matchesPlayed)
            matchesPlayed += 1 #just to prevent spam
        if queue.size == 0:
            queue = np.append(queue, random.choice(playerArr))
            pass
        if queue.size> maxQueueSize:
            maxQueueSize = queue.size
            
        else:
            newPlayer = random.choice(playerArr)
            addPos = np.searchsorted(queue, newPlayer)
            if newPlayer in queue:
                pass #can't play against themself
            else:
                addPos = np.searchsorted(queue, newPlayer)
                try:
                    opponent = queue[addPos]
                except IndexError as error: #always occurs at the end of the list. 
                    queue = np.insert(queue, addPos, newPlayer)
                    pass 
                try: 
                    if allowMatch(newPlayer, opponent, mode = mode, cardLvlRule = cardLvlRule, KTdiff = KTdiff, KTcutoff = KTcutoff):
                            newPlayer.playMatch(opponent)
                            matchesPlayed += 1
                            queue = np.delete(queue, addPos)
                    else:
                             queue = np.insert(queue, addPos, newPlayer)  
                except UnboundLocalError as error:
                    print("Error")
    playerArr.sort()
    return playerArr

In [8]:
def KTsim(playerArr, numMatches, KTdiff = 1, KTcutoff = 6000):
    """Simulates a king tower matchmaking system, but uses separate queues to optimize performance
    Args: 
    playerArr: Array of player objects, sorted by trophies
    numMatches: number of matches to be played
    KTdiff: The difference in king tower between 2 players matched against each other
    ktCutoff:  The point where king tower matchmaking ends"""
    queueDict = {}
    for kt in range(8, 15):
        queueDict[str(kt)] = np.asarray([]) #separated queue for players under cutoff (MUCH FASTER)
    if max(playerArr).trophies >= KTcutoff:
        generalQueue = np.asarray([max(playerArr)]) #general queue for players over the cutoff
    else:
        generalQueue = np.asarray([])
    matchesPlayed = 0
    #cardRule = lambda p1, p2: abs(p1.cardLevel - p2.cardLevel) < cardLvlRule
    kTDiffRule = lambda p1, p2: abs(p1.kt - p2.kt) <= KTdiff
    ktCutoffRule = lambda p: p.trophies > KTcutoff
    while matchesPlayed < numMatches:
        if matchesPlayed %(numMatches//10) == 0:
            print(matchesPlayed)
        newPlayer = random.choice(playerArr)
        if newPlayer.trophies > KTcutoff:
            addPos= np.searchsorted(generalQueue, newPlayer)
            if addPos == generalQueue.size:
                addPos = addPos - 1
            opponent = generalQueue[addPos]
            try: 
                if allowMatch(newPlayer,opponent,mode='KT',KTdiff=KTdiff,KTcutoff=KTcutoff):
                    newPlayer.playMatch(opponent)
                    matchesPlayed += 1
                    generalQueue = np.delete(generalQueue, addPos)
                else:
                    generalQueue = np.insert(generalQueue, addPos, newPlayer)  
            except UnboundLocalError as error:
                print("Error")
        else: #need to use KT queues
            opponent = findOpponent(queueDict, newPlayer, KTdiff, KTcutoff)
            if opponent == newPlayer: #there is no opponents, add to proper queue
                queueDict = addToQueue(queueDict, newPlayer)
            else:
                newPlayer.playMatch(opponent)
                matchesPlayed += 1
                queueDict = remFromQueue(queueDict, opponent)
    playerArr.sort()
    return playerArr
                
            
        

In [9]:
def capByKt(data):
    for p in data:
        if p.cardLevel > 8*p.kt: #cap by King tower
            p.cardLevel = 8*p.kt
    return data


In [10]:
def addToQueue(qDict,player):
    """adds the player to the correct queue in the qDict"""
    key = str(int(player.kt))
    q = qDict[key]
    if q.size == 0:
        qDict[key] = np.append(q, player)
    elif player not in q:
            addPos = np.searchsorted(q, player)
            qDict[key] = np.insert(q, addPos, player)
    return qDict

def remFromQueue(qDict, player):
    """Removes the player from the correct queue in the qDict"""
    key = str(int(player.kt))
    q = qDict[key]
    delPos = np.searchsorted(q, player)
    if delPos == q.size:
        delPos = delPos-1
    qDict[key] = np.delete(q, delPos)
    return qDict

In [11]:
def findOpponent(qDict, p, ktDiff, ktCutoff):
    """Finds an opponent if p.trophies is under the KT threshold. 
    Only call this if ktCutoffRule(p) == False 
    ktDiff >=0 """
    key = int(p.kt) #ensure int, not float
    orderToCheck = [str(key)]
    for diff in range(1, ktDiff+1):
        if 8 <= key + diff <= 14:
            orderToCheck += [str(key + diff)]
        if 8<= key - diff <= 14:
            orderToCheck += [str(key - diff)]
    for ktQueue in orderToCheck:
        q = qDict[ktQueue]
        addPos = np.searchsorted(q, p)
        if q.size == 0:
            pass
        elif addPos == q.size:
            opponent = q[addPos-1]
        else:
            opponent = q[addPos]
        try:
            if allowMatch(p, opponent, mode = 'KT', KTdiff = ktDiff, KTcutoff = ktCutoff):
                return opponent
        except UnboundLocalError as error:
            pass
    return p
    

In [12]:
def allowMatch(p1, p2, mode = None, cardLvlRule = 100, KTdiff = 0, KTcutoff = 5000):
    cardRule = lambda p1, p2: abs(p1.cardLevel - p2.cardLevel) < cardLvlRule
    KTDiffRule = lambda p1, p2: abs(p1.kt - p2.kt) <= KTdiff
    KtCutoffRule = lambda p: p.trophies > KTcutoff
    if p1.matchAllowed(p2) == False:
        return False
    elif mode == 'CL':
        return cardRule(p1, p2)
    elif mode == 'KT':
        return KTDiffRule(p1, p2) or p1.trophies > KTcutoff 
    elif mode == 'KTCL':
        return (KTDiffRule(p1, p2) or p1.trophies > KTcutoff) and cardRule(p1, p2)
    else:
        return True
        

In [13]:
def plots(data):
    """Makes n plots from the data:
    Bar graph of Lvl diff/Match vs King Tower
    Histogram of Trophies and King Tower
    
    """
    df = arrToDF(data)
    df2 = arrToDF(sepByCards(data))
    plt.figure(figsize = (6, 6))
    sns.barplot(x = "King Tower", y = "LvlDiff/Match", data = df)
    
    plt.figure(figsize = (20,6))
    sns.barplot(x = "Card Level", y = "LvlDiff/Match", data = df)
    plt.xticks(rotation = -45)
    
    plt.figure(figsize = (20, 8))
    sns.histplot(data = df,
            x = 'Trophies', 
            hue = 'King Tower',
            stat = 'count',
             palette = PALETTE,
             multiple = 'stack')
    plt.figure(figsize = (20, 8))
    sns.histplot(data=df2, 
             x = 'Trophies',
             hue = 'Card Level',
             stat = 'count',
             palette = sns.color_palette("CMRmap_r", n_colors = 13),
             multiple = 'stack')
    
    plt.figure(figsize = (16, 8))    
    sns.relplot(data = df, x = "Trophies", y = "Card Level", hue = "King Tower", palette = PALETTE)
    
    plt.figure(figsize = (20, 8))
    sns.relplot(data = df, x='Trophies', y = 'LvlDiff/Match', hue = 'King Tower', palette = PALETTE)

In [14]:
def sepByCards(arr):
    """Separates an array of player objecs by cards
    Args:
    arr: Array of player objects
    """
    playerCopy = [] #need to deepcopy the array
    for p in arr:
            new = pl.Player(id = p.id,
                            trophies=p.trophies,
                            wins=p.wins,
                            losses=p.losses,
                            kingLevel=p.kt,
                            cardLevel=p.cardLevel, 
                            totalLvlDiff=p.totalLvlDiff)
            playerCopy += [new]
    newArr = np.asarray(playerCopy)
    for player in newArr:
        if 60 <= player.cardLevel <= 63:
            player.cardLevel = '60-63'
        elif 64 <= player.cardLevel <= 67:
            player.cardLevel = '64-67'
        elif 68 <= player.cardLevel <= 71:
            player.cardLevel = '68-71'
        elif 72 <= player.cardLevel <= 75:
            player.cardLevel = '72-75'
        elif 76 <= player.cardLevel <= 79:
            player.cardLevel = '76-79'
        elif 80 <= player.cardLevel <= 83:
            player.cardLevel = '80-83'
        elif 84 <= player.cardLevel <= 87:
            player.cardLevel = '84-87'
        elif 88 <= player.cardLevel <= 91:
            player.cardLevel = '88-91'
        elif 92 <= player.cardLevel <= 95:
            player.cardLevel = '92-95'
        elif 96 <= player.cardLevel <= 99:
            player.cardLevel = '96-99'
        elif 100 <= player.cardLevel <= 103:
            player.cardLevel = '100-103'
        elif 104 <= player.cardLevel <= 107:
            player.cardLevel = '104-107'
        elif 108 <= player.cardLevel:
            player.cardLevel = '108-112'
    return newArr

In [15]:
def arrToDF(arr):
    """Converts an array of player objects to a dataframe
    Args: arr- Array of Player objects
    Returns: dataframe. 
    """
    dataInLists = [p.getData() + [(p.totalLvlDiff/(p.wins+p.losses))] for p in arr]
    return pd.DataFrame(data = dataInLists, columns = COLS)
     
def storeDF(df, filename):
    """Stores the dataframe in csv file filename
    Args:
    df-  Dataframe to store
    filename: string, name of file"""
    df.to_csv(filename, index_label = False)
    
def dfToArr(df, reset = False):
    """Converts a dataframe back into a numpy array of player objects"""
    arrays = df.to_numpy()
    playerList = []
    if reset:
        for p in arrays:
            new = pl.Player(id = p[0], trophies = p[1],wins=0,losses =0, kingLevel=p[4],cardLevel = p[5],totalLvlDiff=0)
            new.reset()
            playerList += [new]
    else:
        for p in arrays:
            new = pl.Player(id = p[0],trophies=p[1],wins=p[2],losses=p[3],kingLevel=p[4],cardLevel=p[5], totalLvlDiff=p[6])
            playerList += [new]
    return np.asarray(playerList)
    

In [16]:
baselineDF = pd.read_csv('baselineData.csv')
inf6000DF = pd.read_csv('noRules.csv')
currentDF = pd.read_csv('currentLadder.csv')

In [17]:
arr = dfToArr(baselineDF, reset = True)

In [18]:
help(KTsim)
help(continueSim)

Help on function KTsim in module __main__:

KTsim(playerArr, numMatches, KTdiff=1, KTcutoff=6000)
    Simulates a king tower matchmaking system, but uses separate queues to optimize performance
    Args: 
    playerArr: Array of player objects, sorted by trophies
    numMatches: number of matches to be played
    KTdiff: The difference in king tower between 2 players matched against each other
    ktCutoff:  The point where king tower matchmaking ends

Help on function continueSim in module __main__:

continueSim(playerArr, numMatches, mode=None, cardLvlRule=0, KTdiff=0, KTcutoff=5000)
    Continues a simulation for numMatches more games, Very slow if using mode isn't none
    Args: 
    playerArr: Array of player objects.  
    numMatches: Number of matches to be played
    mode: The type of matchmaking to use.  Either 'KT', 'CL', 'KTCL'
    cardLvlRule: Int, the maximum difference in card levels allowed in a match
    KTdiff: The maximum difference in king tower between both players
